In [1]:
# pip install dotenv
# %pip install pyTelegramBotAPI
# %pip install git+https://github.com/huggingface/transformers.git
# %pip install accelerate


In [2]:
import os
from dotenv import load_dotenv
import telebot
import torch
from transformers import pipeline

In [3]:
# Access the BOT_TOKEN variable
BOT_TOKEN = "7816728270:AAEdniunAEPJFwlFeFjXWWikYtEOxXh5bSE"
bot = telebot.TeleBot(BOT_TOKEN)

In [4]:
@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, "hello, how can i help?")

pipe = pipeline(
    "text-generation",
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    torch_dtype=torch.bfloat16,
    device=0 if torch.cuda.is_available() else -1 
)

@bot.message_handler(func=lambda msg: True)
def echo_all(message):
    input= [
            {"role": "system", "content": "You are a chatbot who responds in the style of a helper"},
            {"role": "user", "content": message.text},
        ]
    prompt = pipe.tokenizer.apply_chat_template(
        input,
        tokenize=False,
        add_generation_prompt=True
    )
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)

    generated_text = outputs[0]["generated_text"]
    assistant_response = generated_text[len(prompt):].strip()

    bot.reply_to(message, assistant_response)

# Run the bot
bot.infinity_polling()

Device set to use cuda:0
2025-01-26 19:33:05,231 (__init__.py:1121 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
2025-01-26 19:33:05,234 (__init__.py:1123 MainThread) ERROR - TeleBot: "Break infinity polling"
